In [5]:
import scipy.io as sio
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from scipy.stats.stats import pearsonr as ps
import math
import scipy.sparse.linalg as SLA

In [6]:
# load data
nP = 9 
nTrial = 360
nWord = 60
nRep = 6
nFeature = 25

contents = []
data = []
info = []
meta = []
for i in range(nP):
    contents.append(sio.loadmat('./fMRI_data/data-science-P'+str(i+1)+'.mat'))
    data.append(contents[i]['data'])
    info.append(contents[i]['info'])
    meta.append(contents[i]['meta'])

In [7]:
X = np.array(pd.read_excel('./fMRI_data/features4.xls'))
X = X.T
X = X * np.divide(1, np.linalg.norm(X, axis = 1))[:, np.newaxis]

In [8]:
words = ['airplane', 'ant', 'apartment', 'arch', 'arm', 'barn', 'bear', 'bed', 'bee', 'beetle', \
         'bell', 'bicycle', 'bottle', 'butterfly', 'car', 'carrot', 'cat', 'celery', 'chair', \
         'chimney', 'chisel', 'church', 'closet', 'coat', 'corn', 'cow', 'cup', 'desk', 'dog', \
         'door', 'dress', 'dresser', 'eye', 'fly', 'foot', 'glass', 'hammer', 'hand', 'horse', \
         'house', 'igloo', 'key', 'knife', 'leg', 'lettuce', 'pants', 'pliers', 'refrigerator', \
         'saw', 'screwdriver', 'shirt', 'skirt', 'spoon', 'table', 'telephone', 'tomato', \
         'train', 'truck', 'watch', 'window']

# Classify same words
word_index = [[] for k in range(nWord)]
for word in words:
    for j in range(nTrial):
        if info[0][0][j][2][0] == word:
            word_index[words.index(word)].append(j)

counts = []
for i in range(nP):
# for i in [7]:
    
    nVoxel = len(data[i][0][0][0])
    
    Y_cub=np.zeros((nWord, nRep, nVoxel))
    Y = np.zeros((nWord, nVoxel))
    
    # Classify same words
    count = 0
    count2 = 0
    # Average same words
    for k in range(nWord):
        for m in range(nRep):
            Y[k] = Y[k]+data[i][word_index[k][m]][0][0]
            Y_cub[k][m]=data[i][word_index[k][m]][0][0]
        Y[k] = Y[k]/nRep
        
    # Begin bootstrap
    for k in range(0,nWord-1):
        for kk in range(k+1, nWord):
            count2 += 1
        # for kk in range(k+1, k+2):
            X_tr = np.concatenate((X[0:k],X[k+1:kk],X[kk+1:]), axis = 0)
            Y_tr = np.concatenate((Y[0:k],Y[k+1:kk],Y[kk+1:]), axis = 0)
            
            Xmean_tr = np.mean(X_tr,axis=0)
            Ymean_tr = np.mean(Y_tr,axis=0)
            
            X_tr -= Xmean_tr
            Y_tr -= Ymean_tr
            
            # X_va = np.concatenate((X[k].reshape(1,nFeature),X[kk].reshape(1,nFeature)), axis = 0)
            X_va = np.concatenate((X[k].reshape(1,nFeature),X[kk].reshape(1,nFeature)), axis = 0) - Xmean_tr
            
            # Y_va = np.concatenate((Y[k].reshape(1,nVoxel),Y[kk].reshape(1,nVoxel)), axis = 0)
            Y_va = np.concatenate((Y[k].reshape(1,nVoxel),Y[kk].reshape(1,nVoxel)), axis = 0) - Ymean_tr
            
            W = np.linalg.solve(np.dot(X_tr.T,X_tr), np.dot(X_tr.T,Y_tr))
            Y_pred = np.dot(X_va,W)
            
            # cosine similarity voxel selection
            corre=[]
            Y_cos = np.concatenate((Y_cub[0:k],Y_cub[k+1:kk],Y_cub[kk+1:]),axis=0)
            for ivox in range(nVoxel):
                corr_ivox=np.corrcoef(Y_cos[:,:,ivox].T)
                corr_aver=(np.sum(corr_ivox)-6)/30        
                corre.append(corr_aver)
            ind=np.argsort(np.array(corre))[-500:]

                        
            # cosine similarity
            match1122 = cosine_similarity(Y_pred[0][ind].reshape(1,-1), Y_va[0][ind].reshape(1,-1)) + \
                        cosine_similarity(Y_pred[1][ind].reshape(1,-1), Y_va[1][ind].reshape(1,-1))
            match1221 = cosine_similarity(Y_pred[0][ind].reshape(1,-1), Y_va[1][ind].reshape(1,-1)) + \
                        cosine_similarity(Y_pred[1][ind].reshape(1,-1), Y_va[0][ind].reshape(1,-1))
            if match1122 >= match1221: count += 1
    # counts.append(2*count/((nWord-1)*nWord))
    print(count/count2)
    counts.append(count/count2)

0.8152542372881356
0.8152542372881356
0.7531073446327684
0.7361581920903955
0.7180790960451977
0.619774011299435
0.7141242937853107
0.6146892655367232
0.5689265536723164


In [9]:
np.mean(counts)

0.7061519146264909

In [10]:
count2

1770